<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Level-Steps" data-toc-modified-id="Level-Steps-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Level Steps</a></span></li><li><span><a href="#Create-DataFrame" data-toc-modified-id="Create-DataFrame-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create DataFrame</a></span><ul class="toc-item"><li><span><a href="#Focus-group" data-toc-modified-id="Focus-group-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Focus group</a></span></li><li><span><a href="#Clean-columns" data-toc-modified-id="Clean-columns-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Clean columns</a></span></li><li><span><a href="#Remove-duplicates" data-toc-modified-id="Remove-duplicates-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Remove duplicates</a></span></li><li><span><a href="#Reindex" data-toc-modified-id="Reindex-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Reindex</a></span></li><li><span><a href="#Hierarchy-file-reading:-upto-disagg-5" data-toc-modified-id="Hierarchy-file-reading:-upto-disagg-5-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Hierarchy file reading: upto disagg 5</a></span><ul class="toc-item"><li><span><a href="#Reading-files" data-toc-modified-id="Reading-files-3.5.1"><span class="toc-item-num">3.5.1&nbsp;&nbsp;</span>Reading files</a></span></li><li><span><a href="#Cleaning" data-toc-modified-id="Cleaning-3.5.2"><span class="toc-item-num">3.5.2&nbsp;&nbsp;</span>Cleaning</a></span></li><li><span><a href="#Max-disagg-levels" data-toc-modified-id="Max-disagg-levels-3.5.3"><span class="toc-item-num">3.5.3&nbsp;&nbsp;</span>Max disagg levels</a></span></li></ul></li><li><span><a href="#Get-the-highest-disaggregation-level-for-every-record" data-toc-modified-id="Get-the-highest-disaggregation-level-for-every-record-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Get the highest disaggregation level for every record</a></span></li><li><span><a href="#Get-separate-tag-columns-for-each-disagg-level" data-toc-modified-id="Get-separate-tag-columns-for-each-disagg-level-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Get separate tag columns for each disagg level</a></span></li><li><span><a href="#Sanity-Check-1:-Nulls" data-toc-modified-id="Sanity-Check-1:-Nulls-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>Sanity Check 1: Nulls</a></span></li><li><span><a href="#Sanity-Check-2:-Tags-inconsistent-between-hierarchy-and-current-data" data-toc-modified-id="Sanity-Check-2:-Tags-inconsistent-between-hierarchy-and-current-data-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>Sanity Check 2: Tags inconsistent between hierarchy and current data</a></span></li><li><span><a href="#Sanity-Check-3:-broad-cats-that-are-present" data-toc-modified-id="Sanity-Check-3:-broad-cats-that-are-present-3.10"><span class="toc-item-num">3.10&nbsp;&nbsp;</span>Sanity Check 3: broad cats that are present</a></span></li></ul></li><li><span><a href="#Reindex" data-toc-modified-id="Reindex-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Reindex</a></span></li><li><span><a href="#Export-dataframe" data-toc-modified-id="Export-dataframe-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Export dataframe</a></span></li></ul></div>

To do: add higher disaggregations

Note, this version has broadcat joined using tags

# Libraries

In [1]:
import pandas as pd  #dataframe
import numpy as np  #stats
import pickle  #loading data
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
import re  #regex

from sklearn.model_selection import cross_val_score  # cross validation
from sklearn.model_selection import StratifiedKFold # cross validation

from sklearn.model_selection import train_test_split  #train_test_split
from sklearn.feature_extraction.text import CountVectorizer  #vectorizer
from sklearn.feature_extraction.text import TfidfTransformer  #tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  #model

from sklearn.pipeline import Pipeline  # Pipeline

import string  # for punctuations

import matplotlib.pyplot as plt  # visualization
import seaborn as sns  #  visualization

from scipy.sparse import csr_matrix  # dealing with sparse matrix
from scipy.sparse import hstack  # joining sparse matrix
from scipy.sparse import vstack  # joining sparse matrix

from sklearn.metrics import confusion_matrix #metrics
from sklearn import metrics #metrics


import re #regex
from wordcloud import WordCloud #wordcloud
import nltk # for stopwords
nltk.download('stopwords') #stopwords
from nltk.corpus import stopwords #stopwords
from pprint import pprint #lda
from nltk.stem.snowball import SnowballStemmer # for stemming

import spacy # for lemmatization
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gbatt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Level Steps

Data Preprocessing performed in notebook Modeling_with_loc_lvl2_v2. We use the dataframe generated as df_with_predicted_broad_cat_v2.sav for proceeding to next levels of disaggregations

# Create DataFrame

## Focus group

In [53]:
# Code to retrieve pickle
# use pickle to retrieve

file = open("df_with_predicted_broad_cat_v3.sav",'rb')
df = pickle.load(file)
file.close()

In [54]:
# Rename y_actual and y_pred

df.rename(columns = {"y_actual": "y_actual_broad_cat",
                    "y_pred": "y_pred_broad_cat"}, inplace = True)



In [55]:
# Filter for OpExp
df_focus = df.copy()

In [56]:
# check shape
df_focus.shape

(1117040, 22)

In [57]:
df_focus.columns

Index(['Unnamed: 0', 'adsh', 'report', 'line', 'stmt', 'inpth', 'tag',
       'version_x', 'prole', 'plabel', 'negating', 'cik', 'name', 'sic',
       'form', 'aggregation_level', 'broad_cat', 'version_y', 'plabel_proc',
       'y_actual_broad_cat', 'y_pred_broad_cat', 'broad_cat_pred'],
      dtype='object')

## Clean columns

In [58]:
df_focus

,Unnamed: 0,adsh,report,line,stmt,inpth,tag,version_x,prole,plabel,...,name,sic,form,aggregation_level,broad_cat,version_y,plabel_proc,y_actual_broad_cat,y_pred_broad_cat,broad_cat_pred
0,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,"GLOBAL WATER RESOURCES, INC.",4941.0,10-K,1,IntExp,us-gaap/2018,write off of debt issuance costs,0,0,IntExp
1,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,"GLOBAL WATER RESOURCES, INC.",4941.0,10-K,1,IntExp,us-gaap/2019,write off of debt issuance costs,0,0,IntExp
2,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,"GLOBAL WATER RESOURCES, INC.",4941.0,10-K,1,IntExp,us-gaap/2020,write off of debt issuance costs,0,0,IntExp
3,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,"GLOBAL WATER RESOURCES, INC.",4941.0,10-K,1,IntExp,us-gaap/2021,write off of debt issuance costs,0,0,IntExp
4,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,"GLOBAL WATER RESOURCES, INC.",4941.0,10-K,1,IntExp,us-gaap/2022,write off of debt issuance costs,0,0,IntExp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117035,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,"GOODNESS GROWTH HOLDINGS, INC.",2834.0,10-Q,1,OpExp,us-gaap/2018,accretion,3,3,OpExp
1117036,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,"GOODNESS GROWTH HOLDINGS, INC.",2834.0,10-Q,1,OpExp,us-gaap/2019,accretion,3,3,OpExp
1117037,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,"GOODNESS GROWTH HOLDINGS, INC.",2834.0,10-Q,1,OpExp,us-gaap/2020,accretion,3,3,OpExp
1117038,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,"GOODNESS GROWTH HOLDINGS, INC.",2834.0,10-Q,1,OpExp,us-gaap/2021,accretion,3,3,OpExp


Keeping all columns so each record is a line item

In [112]:
# # Remove unneeded columns
# df_focus.drop(["adsh", "report", "line", "stmt", "inpth", "version_x", "prole", "plabel", "version_y" ,"negating", "cik","sic", "name", "form",
#               "Revenue_max_locdum", "COGS_max_locdum", "OpInc_max_locdum"], axis = 1, inplace = True)

In [59]:
# columns we are working with
df_focus.columns

Index(['Unnamed: 0', 'adsh', 'report', 'line', 'stmt', 'inpth', 'tag',
       'version_x', 'prole', 'plabel', 'negating', 'cik', 'name', 'sic',
       'form', 'aggregation_level', 'broad_cat', 'version_y', 'plabel_proc',
       'y_actual_broad_cat', 'y_pred_broad_cat', 'broad_cat_pred'],
      dtype='object')

## Remove duplicates

In [60]:
# check duplicates
df_focus.duplicated().sum()

45

In [61]:
# remove duplicates
df_focus.drop_duplicates(inplace = True)

## Reindex

In [62]:
# check duplicated indices
df_focus.index.duplicated().sum()

0

In [63]:
# reindex

df_focus.reset_index(drop = True, inplace = True)

## Hierarchy file reading: upto disagg 5

### Reading files

In [64]:
# set path

path = os.getcwd()
path = path.replace("\\\\", "\\")
path = path + r'/disagg'

In [65]:
os.listdir(path)

['NetCashProvidedByUsedInFinancingActivities_XBRL_disagg.csv',
 'NetCashProvidedByUsedInInvestingActivities_XBRL_disagg.csv',
 'NetCashProvidedByUsedInOperatingActivities_XBRL_disagg.csv']

In [66]:
%%time

# Read hierarchy files and create dataframe

df_hier = pd.DataFrame() # create dataframe to update

for filename in os.listdir(path): # loop through files
    
    if filename.endswith('.csv'):
        df_temp = pd.read_csv(path + "/" + filename)
        
        # sanity check: duplicated
        print("number of duplicates in", filename, ":", df_temp.duplicated().sum())
        
        df_hier = pd.concat([df_hier, df_temp], axis = 0)
        
        


number of duplicates in NetCashProvidedByUsedInFinancingActivities_XBRL_disagg.csv : 0
number of duplicates in NetCashProvidedByUsedInInvestingActivities_XBRL_disagg.csv : 0
number of duplicates in NetCashProvidedByUsedInOperatingActivities_XBRL_disagg.csv : 0
CPU times: total: 31.2 ms
Wall time: 27 ms


In [67]:
# sanity check: duplicates

df_hier.duplicated().sum()

0

In [68]:
# view dataframe

display(df_hier.head())
display(df_hier.info())

,disagg_1,disagg_2,disagg_3,disagg_4,disagg_5,disagg_6,disagg_7,disagg_8,disagg_9
0,NetCashProvidedByUsedInFinancingActivities,CashProvidedByUsedInFinancingActivitiesDiscont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,DistributionMadeToLimitedLiabilityCompanyLLCMe...,NaN,NaN,NaN,NaN,NaN,NaN
2,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,DistributionMadeToLimitedPartnerCashDistributi...,NaN,NaN,NaN,NaN,NaN,NaN
3,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,IncreaseDecreaseInAdvancePaymentsByBorrowersFo...,NaN,NaN,NaN,NaN,NaN,NaN
4,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,IncreaseDecreaseInDeposits,IncreaseDecreaseInDemandDeposits,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 585 entries, 0 to 272
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   disagg_1  585 non-null    object
 1   disagg_2  585 non-null    object
 2   disagg_3  583 non-null    object
 3   disagg_4  534 non-null    object
 4   disagg_5  436 non-null    object
 5   disagg_6  270 non-null    object
 6   disagg_7  111 non-null    object
 7   disagg_8  39 non-null     object
 8   disagg_9  2 non-null      object
dtypes: object(9)
memory usage: 45.7+ KB


None

### Cleaning

In [69]:
# basic cleaning

for col in df_hier.columns:
    df_hier[col] = df_hier[col].str.strip()
    

In [70]:
df_hier

,disagg_1,disagg_2,disagg_3,disagg_4,disagg_5,disagg_6,disagg_7,disagg_8,disagg_9
0,NetCashProvidedByUsedInFinancingActivities,CashProvidedByUsedInFinancingActivitiesDiscont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,DistributionMadeToLimitedLiabilityCompanyLLCMe...,NaN,NaN,NaN,NaN,NaN,NaN
2,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,DistributionMadeToLimitedPartnerCashDistributi...,NaN,NaN,NaN,NaN,NaN,NaN
3,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,IncreaseDecreaseInAdvancePaymentsByBorrowersFo...,NaN,NaN,NaN,NaN,NaN,NaN
4,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInFinancingActivitiesCont...,IncreaseDecreaseInDeposits,IncreaseDecreaseInDemandDeposits,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
268,NetCashProvidedByUsedInOperatingActivities,AdjustmentsToReconcileNetIncomeLossToCashProvi...,PaymentsForProceedsFromOtherDeposits,PaymentsForOtherDeposits,NaN,NaN,NaN,NaN,NaN
269,NetCashProvidedByUsedInOperatingActivities,AdjustmentsToReconcileNetIncomeLossToCashProvi...,PaymentsForProceedsFromOtherDeposits,ProceedsFromOtherDeposits,NaN,NaN,NaN,NaN,NaN
270,NetCashProvidedByUsedInOperatingActivities,AdjustmentsToReconcileNetIncomeLossToCashProvi...,PaymentsForProceedsFromTenantAllowance,NaN,NaN,NaN,NaN,NaN,NaN
271,NetCashProvidedByUsedInOperatingActivities,ProfitLoss,IncomeLossIncludingPortionAttributableToNoncon...,NaN,NaN,NaN,NaN,NaN,NaN


### Max disagg levels

In [71]:
# set max disagg level

max_disagg = 5
#max_disagg = df_hier.shape[1]



In [72]:
# keep records upto disagg max_disagg

df_hier = df_hier.iloc[:,:max_disagg].copy()

Note, after selecting a particular disagg level, there are going to be duplicates in the file. Reason being, for duplicated rows, subsequent columns will have different disaggregated values. But since we cut off the later columns, we now are left with duplicates. We need to remove the same

In [73]:
# Remove duplicates

print("number of duplicates we drop", df_hier.duplicated().sum())

## drop the duplicates
df_hier.drop_duplicates(inplace = True)

number of duplicates we drop 220


In [74]:
# get a list of all unique values across aggregation levels


unique_list = []

for i in range(1, max_disagg+1):
    print(i)
    unique_list = unique_list + list(df_hier["disagg_" + str(i)].dropna().unique())
    

    

1
2
3
4
5


In [75]:
### Export dataframe

object_filename = "df_hierarchy.sav"
saved_df = pickle.dump(df_hier, open(object_filename,'wb'))
print('Dataframe is saved into to disk successfully Using Pickle')

Dataframe is saved into to disk successfully Using Pickle


**Stepwise Explaination:-**

- first of all get an aggregation level column, that is needed for simplicity and interpretability
- then create column for every disagg level. So for lvl 5, see if the aggregation level is 5, and only keep the value for those records. For lvl 4, keep aggregation level 4 and 5, then convert 5 to 4....and so on
- then we need to check for aggregation level and then decide if we select those records or not


## Get the highest disaggregation level for every record

In [76]:

# loop through disaggregation levels

max_disagg = 5

for i in range(1, max_disagg+1):
    print(i)
    disagg_vals = list(df_hier["disagg_" + str(i)].unique())
    df_focus.loc[df_focus["tag"].isin(disagg_vals), "disagg_level"] = i




1
2
3
4
5


In [77]:
df_focus

,Unnamed: 0,adsh,report,line,stmt,inpth,tag,version_x,prole,plabel,...,sic,form,aggregation_level,broad_cat,version_y,plabel_proc,y_actual_broad_cat,y_pred_broad_cat,broad_cat_pred,disagg_level
0,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2018,write off of debt issuance costs,0,0,IntExp,NaN
1,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2019,write off of debt issuance costs,0,0,IntExp,NaN
2,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2020,write off of debt issuance costs,0,0,IntExp,NaN
3,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2021,write off of debt issuance costs,0,0,IntExp,NaN
4,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2022,write off of debt issuance costs,0,0,IntExp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116990,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2018,accretion,3,3,OpExp,NaN
1116991,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2019,accretion,3,3,OpExp,NaN
1116992,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2020,accretion,3,3,OpExp,NaN
1116993,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2021,accretion,3,3,OpExp,NaN


In [78]:
# sanity check: see if any values are missed by checking NAs in disagg_level

print("sanity check sum (should be 0):", df_focus[df_focus["disagg_level"].isna()]["tag"].isin(unique_list).sum())

sanity check sum (should be 0): 0


In [79]:
df_focus

,Unnamed: 0,adsh,report,line,stmt,inpth,tag,version_x,prole,plabel,...,sic,form,aggregation_level,broad_cat,version_y,plabel_proc,y_actual_broad_cat,y_pred_broad_cat,broad_cat_pred,disagg_level
0,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2018,write off of debt issuance costs,0,0,IntExp,NaN
1,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2019,write off of debt issuance costs,0,0,IntExp,NaN
2,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2020,write off of debt issuance costs,0,0,IntExp,NaN
3,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2021,write off of debt issuance costs,0,0,IntExp,NaN
4,1884297,0001434728-19-000051,7,6,CF,0,WriteOffOfDeferredDebtIssuanceCost,us-gaap/2018,terseLabel,write off of debt issuance costs,...,4941.0,10-K,1,IntExp,us-gaap/2022,write off of debt issuance costs,0,0,IntExp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116990,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2018,accretion,3,3,OpExp,NaN
1116991,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2019,accretion,3,3,OpExp,NaN
1116992,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2020,accretion,3,3,OpExp,NaN
1116993,12827821,0001558370-22-013196,6,21,CF,0,AccretionExpense,us-gaap/2021,terseLabel,accretion,...,2834.0,10-Q,1,OpExp,us-gaap/2021,accretion,3,3,OpExp,NaN


In [80]:
# Drop records where there is no disagg level

# check percent of data we are working with currently
print("data percent we dropping", 100*df_focus["disagg_level"].isna().sum()/len(df_focus))
print("number of rows we are working with", df_focus["disagg_level"].notnull().sum())

df_focus.dropna(subset = ["disagg_level"], inplace = True)

data percent we dropping 50.22761964019534
number of rows we are working with 555955


## Get separate tag columns for each disagg level

In [81]:
# intermediate step copy

df_new = df_focus.copy()

# set max disaggregation level
max_disagg = df_hier.shape[1]


In [82]:
for current_disagg in range(max_disagg, 0, -1):
    print(current_disagg)
    
    # make copy of relevant tags
    df_new.loc[df_new["disagg_level"] >= current_disagg, "tag_" + str(current_disagg)] = \
    df_new.loc[df_new["disagg_level"] >= current_disagg, "tag"].copy()
    
    # condition if max
    if current_disagg != max_disagg:
        
        # convert tags to the relevant disagg level
        for j in range(max_disagg, current_disagg, -1 ):
            print("*", j)
            df_mapping = df_hier[df_hier["disagg_" + str(j)].notnull()]
            df_new["tag_" + str(current_disagg)].\
            replace(df_mapping.set_index("disagg_" + str(j))["disagg_" + str(j-1)], inplace = True)


5
4
* 5
3
* 5
* 4
2
* 5
* 4
* 3
1
* 5
* 4
* 3
* 2


## Sanity Check 1: Nulls

In [83]:
# tag 1
df_new[df_new["tag_1"].isna()]["tag"].isin(df_hier["disagg_1"].unique()).sum()

0

In [84]:
# tag 2
df_new[df_new["tag_2"].isna()]["tag"].isin(df_hier["disagg_2"].unique()).sum()

0

In [85]:
# tag 3
df_new[df_new["tag_3"].isna()]["tag"].isin(df_hier["disagg_3"].unique()).sum()

0

In [86]:
# tag 4
df_new[df_new["tag_4"].isna()]["tag"].isin(df_hier["disagg_4"].unique()).sum()

0

In [87]:
# tag 5
df_new[df_new["tag_5"].isna()]["tag"].isin(df_hier["disagg_5"].unique()).sum()

0

## Sanity Check 2: Tags inconsistent between hierarchy and current data

Result should return empty set or nan

In [88]:
# tag 1
set(df_new["tag_1"].unique())  - set(df_new["tag"].unique())

{'NetCashProvidedByUsedInOperatingActivities'}

In [89]:
# tag 2
set(df_new["tag_2"].unique())  - set(df_new["tag"].unique())

{'AdjustmentsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivities'}

In [90]:
# tag 3
set(df_new["tag_3"].unique())  - set(df_new["tag"].unique())

{'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivities',
 nan}

In [45]:
# tag 4
set(df_new["tag_4"].unique())  - set(df_new["tag"].unique())

{'DeferredIncomeTaxesAndTaxCredits',
 'DepreciationDepletionAndAmortization',
 'EnvironmentalExpenseAndLiabilities',
 'GainLossOnSaleOfLoansAndLeases',
 'IncomeLossFromEquityMethodInvestmentsNetOfDividendsOrDistributions',
 'RestructuringCostsAndAssetImpairmentCharges',
 nan}

In [46]:
# tag 5
set(df_new["tag_5"].unique())  - set(df_new["tag"].unique())

{nan}

**NOTE: If inconsistency exists, modeling won't take an impact. Although, possible to have values in final prediction that don't match original tags. Make sure to adjust for same**

## Sanity Check 3: broad cats that are present

In [91]:
df_new["broad_cat"].value_counts()

OpExp            183810
Profitability    147490
Tax              105250
NonOp             59130
IntExp            56505
Revenues           3770
Name: broad_cat, dtype: int64

In [92]:
df_new["tag_1"].value_counts()

NetCashProvidedByUsedInOperatingActivities    555955
Name: tag_1, dtype: int64

# Reindex

we reindex since we dropped data again (where disagg_level is null again)

In [93]:
# check for duplicates and duplicated indices

display(df_new.duplicated().sum())
display(df_new.index.duplicated().sum())


0

0

In [94]:
# sort and then reset index: in c
df_new = df_new.sort_index().reset_index(drop = True)

In [95]:
df_new.head()

,Unnamed: 0,adsh,report,line,stmt,inpth,tag,version_x,prole,plabel,...,plabel_proc,y_actual_broad_cat,y_pred_broad_cat,broad_cat_pred,disagg_level,tag_5,tag_4,tag_3,tag_2,tag_1
0,15014272,0001564590-19-041725,6,10,CF,0,ResearchAndDevelopmentInProcess,us-gaap/2019,terseLabel,acquired ipr d,...,acquired ipr,3,3,OpExp,4.0,NaN,ResearchAndDevelopmentInProcess,AdjustmentsNoncashItemsToReconcileNetIncomeLos...,AdjustmentsToReconcileNetIncomeLossToCashProvi...,NetCashProvidedByUsedInOperatingActivities
1,15014272,0001564590-19-041725,6,10,CF,0,ResearchAndDevelopmentInProcess,us-gaap/2019,terseLabel,acquired ipr d,...,acquired ipr,3,3,OpExp,4.0,NaN,ResearchAndDevelopmentInProcess,AdjustmentsNoncashItemsToReconcileNetIncomeLos...,AdjustmentsToReconcileNetIncomeLossToCashProvi...,NetCashProvidedByUsedInOperatingActivities
2,15014272,0001564590-19-041725,6,10,CF,0,ResearchAndDevelopmentInProcess,us-gaap/2019,terseLabel,acquired ipr d,...,acquired ipr,3,3,OpExp,4.0,NaN,ResearchAndDevelopmentInProcess,AdjustmentsNoncashItemsToReconcileNetIncomeLos...,AdjustmentsToReconcileNetIncomeLossToCashProvi...,NetCashProvidedByUsedInOperatingActivities
3,15014272,0001564590-19-041725,6,10,CF,0,ResearchAndDevelopmentInProcess,us-gaap/2019,terseLabel,acquired ipr d,...,acquired ipr,3,3,OpExp,4.0,NaN,ResearchAndDevelopmentInProcess,AdjustmentsNoncashItemsToReconcileNetIncomeLos...,AdjustmentsToReconcileNetIncomeLossToCashProvi...,NetCashProvidedByUsedInOperatingActivities
4,15014272,0001564590-19-041725,6,10,CF,0,ResearchAndDevelopmentInProcess,us-gaap/2019,terseLabel,acquired ipr d,...,acquired ipr,3,3,OpExp,4.0,NaN,ResearchAndDevelopmentInProcess,AdjustmentsNoncashItemsToReconcileNetIncomeLos...,AdjustmentsToReconcileNetIncomeLossToCashProvi...,NetCashProvidedByUsedInOperatingActivities


# Export dataframe

In [96]:
# export df

object_filename = "df_with_loc_lvl_step_v4.sav"
saved_df = pickle.dump(df_new, open(object_filename,'wb'))
print('Dataframe is saved into to disk successfully Using Pickle')

Dataframe is saved into to disk successfully Using Pickle
